In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    Integer,
    String,
    Float,
    Boolean,
    Date,
    ForeignKey,
)
import re



# TODO: Mettre data constraints pour DATES FIELDS
# TODO: Les FOREIGN KEYS DANS LES SQL QUERIES


pd.set_option("display.max_columns", None)

df = pd.read_csv("glassdoor_dataset/glassdoor.csv")


print(df.dtypes)


# def replace_dots(df):
#     """Replace dots with underscores in column names for better SQL compatibility in queries"""
    
#     df.columns = [col.replace('.', '_') for col in df.columns]
    
#     return df

# df = replace_dots(df)

# # Filtering the main glassdoor.csv file, a lot of columns are HTML or URL related elements that are not very relevant for our purpose.
# filtered_df = df.loc[
#     :,
#     [
#         "gaTrackerData_jobTitle",
#         "gaTrackerData_location",
#         "gaTrackerData_locationType",
#         "gaTrackerData_sector",
#         "header_easyApply",
#         "header_employerName",
#         "header_jobTitle",
#         "header_location",  # Location City (most of times)
#         "header_posted",  # Date job was posted
#         "header_rating",  # Company rating by employees
#         "header_urgencyLabel",  # Misleading column name, it actually indicates the presence of the "2019 Glassdoor Best Place to Work" award on the job posting
#         "header_payHigh",
#         "header_payMed",
#         "header_payLow",
#         "job_description",
#         "job_jobSource",
#         "map_country",  # do something with country_names_2_digit_codes
#         "map_employerName",
#         "map_lat",  # 0 for NaN
#         "map_lng",  # 0 for NaN
#         "map_location",
#         "overview_foundedYear",  # 0 for NaN
#         "overview_hq",
#         "overview_industry",
#         "overview_revenue",
#         "overview_sector",
#         "overview_size",
#         "overview_stock",
#         "overview_type",
#         "overview_description",
#         "overview_mission",
#         "overview_competitors",  # foreign key
#         "rating_ceo_name",
#         "rating_ceoApproval",  # <0 or NaN for missing data
#         "rating_recommendToFriend",  # <0 or NaN for missing data
#         "rating_starRating",
#         "benefits_comments",  # foreign key
#         "benefits_highlights",  # foreign key
#         "reviews",  # foreign key
#         "salary_salaries",  # foreign key
#         "wwfu",  # foreign key
#     ],
# ]


# # Transform columns with floats having zero decimal value into integers
# columns_to_int = [
#     "header_payHigh",
#     "header_payMed",
#     "header_payLow",
#     "overview_foundedYear",
#     "overview_competitors",
#     "benefits_comments",
#     "benefits_highlights",
#     "wwfu",
# ]
# for column in columns_to_int:
#     filtered_df[column] = filtered_df[column].astype(pd.Int64Dtype())


# # Replace negative values with NaN
# columns_to_replace_neg = [
#     "header_rating",
#     "rating_starRating",
#     "rating_ceoApproval",
#     "rating_recommendToFriend",
# ]
# for column in columns_to_replace_neg:
#     filtered_df[column] = filtered_df[column].apply(lambda x: np.nan if x < 0 else x)

# # Replace zero values with NaN
# columns_to_replace_zero = [
#     "map_lat",
#     "map_lng",
# ]
# for column in columns_to_replace_zero:
#     filtered_df[column] = filtered_df[column].apply(lambda x: np.nan if x == 0 else x)

# # Replace outlier/erroneous low salaries values with NaN
# columns_to_replace_outlier_salaries = [
#     "header_payHigh",
#     "header_payMed",
#     "header_payLow",
#     "salary_salaries",
# ]
# for column in columns_to_replace_outlier_salaries:
#     filtered_df[column] = filtered_df[column].apply(lambda x: np.nan if pd.isna(x) or x < 10000 else x)

# # Replace outlier/erroneous years values with NaN
# columns_to_replace_outlier_years = [
#     "overview_foundedYear",
# ]
# for column in columns_to_replace_outlier_years:
#     filtered_df[column] = filtered_df[column].apply(lambda x: np.nan if x < 1000 else x)

# # Remove only rows with all missing values
# filtered_df.dropna(how="all", inplace=True)

# # Remove duplicates
# filtered_df.drop_duplicates(inplace=True)

# def clean_job_description(text):
#     """Removes all the HTML/CSS tags and other random junk from the text, only keeping words"""
    
#     if text is None or pd.isna(text):
#         return text

#     # Remove HTML tags
#     text = re.sub("<[^>]+>", "", text)

#     # Remove HTML entities
#     text = re.sub("&\w+;|&#\d+;", "", text)

#     # Convert text to lowercase
#     text = text.lower()

#     # Remove unwanted characters
#     text = re.sub("[^\w\s]+", " ", text)

#     # Remove extra spaces
#     text = re.sub("\s+", " ", text)

#     # Strip leading/trailing spaces
#     text = text.strip()

#     return text

# filtered_df["job_description"] = filtered_df["job_description"].apply(clean_job_description)

# print(f"dtypes : \n{filtered_df.dtypes}\n\n")
# print(
#     f"Non-NaN percentage : \n{(filtered_df.count()/ len(filtered_df) * 100).round(2)}%"
# )
# print("\n\n\n%%%%%%%%%%%%%%%%%%%%%%%%\n\n\n")
# print(filtered_df.head(60))
# print(filtered_df.columns)
# print(f"There are {len(filtered_df.columns)} columns")
# print(filtered_df.describe())


# create_table_glassdoor = '''
# CREATE TABLE glassdoor (
#     id SERIAL PRIMARY KEY,
#     gaTrackerData_jobTitle VARCHAR,
#     gaTrackerData_location VARCHAR,
#     gaTrackerData_locationType VARCHAR,
#     gaTrackerData_sector VARCHAR,
#     header_easyApply BOOLEAN,
#     header_employerName VARCHAR,
#     header_jobTitle VARCHAR,
#     header_location VARCHAR,
#     header_posted VARCHAR,
#     header_rating REAL,
#     header_urgencyLabel VARCHAR,
#     header_payHigh INTEGER,
#     header_payMed INTEGER,
#     header_payLow INTEGER,
#     job_description VARCHAR,
#     job_jobSource VARCHAR,
#     map_country VARCHAR,
#     map_employerName VARCHAR,
#     map_lat REAL,
#     map_lng REAL,
#     map_location VARCHAR,
#     overview_foundedYear INTEGER,
#     overview_hq VARCHAR,
#     overview_industry VARCHAR,
#     overview_revenue VARCHAR,
#     overview_sector VARCHAR,
#     overview_size VARCHAR,
#     overview_stock VARCHAR,
#     overview_type VARCHAR,
#     overview_description VARCHAR,
#     overview_mission VARCHAR,
#     overview_competitors INTEGER,
#     rating_ceo_name VARCHAR,
#     rating_ceoApproval REAL,
#     rating_recommendToFriend REAL,
#     rating_starRating REAL,
#     benefits_comments INTEGER,
#     benefits_highlights INTEGER,
#     reviews INTEGER,
#     salary_salaries REAL,
#     wwfu INTEGER,
#     FOREIGN KEY (overview_competitors) REFERENCES glassdoor_overview_competitors(id),
#     FOREIGN KEY (benefits_comments) REFERENCES glassdoor_benefits_comments(id),
#     FOREIGN KEY (benefits_highlights) REFERENCES glassdoor_benefits_highlights(id),
#     FOREIGN KEY (reviews) REFERENCES glassdoor_reviews(id),
#     FOREIGN KEY (salary_salaries) REFERENCES glassdoor_salary_salaries(id),
#     FOREIGN KEY (wwfu) REFERENCES glassdoor_wwfu(id)
# );
# '''



# df_oc = pd.read_csv("glassdoor_dataset/glassdoor_overview_competitors.csv")

# replace_dots(df_oc)

# print(df_oc.head(10))
# print(df_oc.describe())
# print(df_oc.dtypes)
# print("\n\n\n\n\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n\n\n\n\n")

# create_table_glassdoor_oc = '''
# CREATE TABLE glassdoor_overview_competitors (
#     id INTEGER PRIMARY KEY,
#     index INTEGER,
#     overview_competitors_val VARCHAR
# );
# '''



# df_bc = pd.read_csv("glassdoor_dataset/glassdoor_benefits_comments.csv")

# df_bc = replace_dots(df_bc)

# # Transform floats with zero decimal values to integers
# columns_to_int = [
#     "index",
#     "benefits_comments_val_rating",
# ]
# for column in columns_to_int:
#     df_bc[column] = df_bc[column].astype(pd.Int64Dtype())

# print(df_bc.head(10))
# print(df_bc.describe())
# print(df_bc.dtypes)
# print("\n\n\n\n\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n\n\n\n\n")

# create_table_glassdoor_bc = '''
# CREATE TABLE glassdoor_benefits_comments (
#     id INTEGER PRIMARY KEY,
#     index INTEGER,
#     benefits_comments_val_city VARCHAR,
#     benefits_comments_val_comment VARCHAR,
#     benefits_comments_val_createDate VARCHAR,
#     benefits_comments_val_currentJob VARCHAR,
#     benefits_comments_val_jobTitle VARCHAR,
#     benefits_comments_val_rating INTEGER,
#     benefits_comments_val_state VARCHAR
# );
# '''



# df_bh = pd.read_csv("glassdoor_dataset/glassdoor_benefits_highlights.csv")

# replace_dots(df_bh)

# # Transform floats with zero decimal values to integers
# columns_to_int = [
#     "index",
#     "benefits_highlights_val_commentCount",
# ]
# for column in columns_to_int:
#     df_bh[column] = df_bh[column].astype(pd.Int64Dtype())


# print(df_bh.head(10))
# print(df_bh.describe())
# print(df_bh.dtypes)
# print("\n\n\n\n\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n\n\n\n\n")

# create_table_glassdoor_bh = '''
# CREATE TABLE glassdoor_benefits_highlights (
#     id INTEGER PRIMARY KEY,
#     benefits_highlights_val_highlightPhrase VARCHAR,
#     benefits_highlights_val_icon VARCHAR,
#     benefits_highlights_val_name VARCHAR,
#     index INTEGER,
#     benefits_highlights_val_commentCount INTEGER
# );

# '''



# df_r = pd.read_csv("glassdoor_dataset/glassdoor_reviews.csv")

# replace_dots(df_r)

# # Transform floats with zero decimal values to integers
# # I found that in some columns, some rare values actually contain decimals (3.5/5 rating for example), I chose to keep those columns as floats
# columns_to_int = [
#     "index",
#     "reviews_val_helpfulCount",
#     "reviews_val_id",
#     # "reviews_val_reviewRatings_careerOpportunities", # Some rare decimals are hidden in this column, keeping as floats
#     # "reviews_val_reviewRatings_compBenefits",  # Some decimals are hidden in this column, keeping as floats
#     "reviews_val_reviewRatings_cultureValues",
#     "reviews_val_reviewRatings_overall", 
#     # "reviews_val_reviewRatings_seniorManagement",  # Some decimals are hidden in this column, keeping as floats
#     # "reviews_val_reviewRatings_worklifeBalance",  # Some decimals are hidden in this column, keeping as floats
#     "reviews_val_summaryPoints_ceoApproval",
#     "reviews_val_summaryPoints_outlook",
#     "reviews_val_summaryPoints_recommend",
#     "reviews_val_reviewResponses",
# ]
# for column in columns_to_int:
#     df_r[column] = df_r[column].astype(pd.Int64Dtype())

# print(df_r.head(20))
# print(df_r.describe())
# print(df_r.dtypes)
# print("\n\n\n\n\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n\n\n\n\n")

# create_table_glassdoor_r = '''
# CREATE TABLE glassdoor_reviews (
#     id INTEGER PRIMARY KEY,
#     index INTEGER,
#     reviews_val_cons VARCHAR,
#     reviews_val_date VARCHAR,
#     reviews_val_featured VARCHAR,
#     reviews_val_helpfulCount INTEGER,
#     reviews_val_id INTEGER,
#     reviews_val_pros VARCHAR,
#     reviews_val_publishedOn VARCHAR,
#     reviews_val_publisher VARCHAR,
#     reviews_val_reviewRatings_careerOpportunities REAL,
#     reviews_val_reviewRatings_compBenefits REAL,
#     reviews_val_reviewRatings_cultureValues INTEGER,
#     reviews_val_reviewRatings_overall INTEGER,
#     reviews_val_reviewRatings_seniorManagement REAL,
#     reviews_val_reviewRatings_worklifeBalance REAL,
#     reviews_val_reviewerDuration VARCHAR,
#     reviews_val_reviewerInformation VARCHAR,
#     reviews_val_reviewerJobTitle VARCHAR,
#     reviews_val_reviewerLocation VARCHAR,
#     reviews_val_reviewerStatus VARCHAR,
#     reviews_val_summaryPoints_ceoApproval INTEGER,
#     reviews_val_summaryPoints_outlook INTEGER,
#     reviews_val_summaryPoints_recommend INTEGER,
#     reviews_val_title VARCHAR,
#     reviews_val_adviceToManagement VARCHAR,
#     reviews_val_companyResponse VARCHAR,
#     reviews_val_reviewResponses INTEGER
# );
# '''



# df_rvr = pd.read_csv("glassdoor_dataset/glassdoor_reviews_val_reviewResponses.csv")

# replace_dots(df_rvr)

# # # Cleaning HTML/CSS tags
# df_rvr["reviews_val_reviewResponses_val_responseText"] = df_rvr["reviews_val_reviewResponses_val_responseText"].apply(clean_job_description)

# print(df_rvr.head(20))
# print(df_rvr.describe())
# print(df_rvr.dtypes)
# print("\n\n\n\n\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n\n\n\n\n")

# create_table_glassdoor_rvr = '''
# CREATE TABLE glassdoor_reviews_val_reviewResponses (
#     id INTEGER PRIMARY KEY,
#     index INTEGER,
#     reviews_val_reviewResponses_val_createDate VARCHAR,
#     reviews_val_reviewResponses_val_helpfulCount INTEGER,
#     reviews_val_reviewResponses_val_jobTitle VARCHAR,
#     reviews_val_reviewResponses_val_notHelpfulCount INTEGER,
#     reviews_val_reviewResponses_val_responseText VARCHAR,
#     reviews_val_reviewResponses_val_responseTextLength INTEGER,
#     reviews_val_reviewResponses_val_totalHelpfulCount INTEGER,
#     reviews_val_reviewResponses_val_updateDate VARCHAR
# );
# '''



# df_ss = pd.read_csv("glassdoor_dataset/glassdoor_salary_salaries.csv")

# replace_dots(df_ss)

# # Transform floats with zero decimal values to integers
# columns_to_int = [
#     "index",
#     "salary_salaries_val_basePayCount",
# ]
# for column in columns_to_int:
#     df_ss[column] = df_ss[column].astype(pd.Int64Dtype())

# print(df_ss.head(60))
# print(df_ss.describe())
# print(df_ss.dtypes)
# print("\n\n\n\n\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n\n\n\n\n")

# create_table_glassdoor_ss = '''
# CREATE TABLE glassdoor_salary_salaries (
#     id INTEGER PRIMARY KEY,
#     index INTEGER,
#     salary_salaries_val_basePayCount INTEGER,
#     salary_salaries_val_jobTitle VARCHAR,
#     salary_salaries_val_payPeriod VARCHAR,
#     salary_salaries_val_salaryPercentileMap_payPercentile10 REAL,
#     salary_salaries_val_salaryPercentileMap_payPercentile90 REAL,
#     salary_salaries_val_salaryPercentileMap_payPercentile50 REAL,
#     salary_salaries_val_salaryType VARCHAR
# );
# '''



# df_w = pd.read_csv("glassdoor_dataset/glassdoor_wwfu.csv")

# replace_dots(df_w)

# # Transform floats with zero decimal values to integers
# columns_to_int = [
#     "wwfu_val_videos",
#     "wwfu_val_photos",
#     "wwfu_val_captions",
# ]
# for column in columns_to_int:
#     df_w[column] = df_w[column].astype(pd.Int64Dtype())

# # Cleaning HTML/CSS tags
# df_w["wwfu_val_body"] = df_w["wwfu_val_body"].apply(clean_job_description)

# print(df_w.head(10))
# print(df_w.describe())
# print(df_w.dtypes)
# print("\n\n\n\n\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n\n\n\n\n")

# create_table_glassdoor_w = '''
# CREATE TABLE glassdoor_wwfu (
#     id INTEGER PRIMARY KEY,
#     index INTEGER,
#     wwf_val_body VARCHAR,
#     wwf_val_id INTEGER,
#     wwf_val_title VARCHAR,
#     wwf_val_type VARCHAR,
#     FOREIGN KEY (wwf_val_videos) INTEGER,
#     wwf_val_photos INTEGER,
#     wwf_val_captions INTEGER
# );
# '''



# df_wvc = pd.read_csv("glassdoor_dataset/glassdoor_wwfu_val_captions.csv")

# replace_dots(df_wvc)


# print(df_wvc.head(10))
# print(df_wvc.describe())
# print(df_wvc.dtypes)
# print("\n\n\n\n\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n\n\n\n\n")

# create_table_glassdoor_wvc = '''
# CREATE TABLE glassdoor_wwfu_val_captions (
#     id INTEGER PRIMARY KEY,
#     index INTEGER,
#     wwfuf_val_captions_val VARCHAR
# );
# '''



# df_wvp = pd.read_csv("glassdoor_dataset/glassdoor_wwfu_val_photos.csv")

# replace_dots(df_wvp)


# print(df_wvp.head(10))
# print(df_wvp.describe())
# print(df_wvp.dtypes)
# print("\n\n\n\n\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n\n\n\n\n")

# create_table_glassdoor_wvp = '''
# CREATE TABLE glassdoor_wwfu_val_photos (
#     id INTEGER PRIMARY KEY,
#     index INTEGER,
#     wwfuf_val_photos_val VARCHAR
# );
# '''



# df_wvv = pd.read_csv("glassdoor_dataset/glassdoor_wwfu_val_videos.csv")

# replace_dots(df_wvv)

# print(df_wvv.head(10))
# print(df_wvv.describe())
# print(df_wvv.dtypes)
# print("\n\n\n\n\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n\n\n\n\n")

# create_table_glassdoor_wvv = '''
# CREATE TABLE glassdoor_wwfu_val_videos (
#     id INTEGER PRIMARY KEY,
#     index INTEGER,
#     wwfuf_val_videos_val VARCHAR
# );
# '''






























# pool, connector = conn_to_psql()


# # Upload the DataFrame to the database using df.to_sql()
# df.to_sql('users', pool, if_exists='replace', index=False)


# filtered_df.to_sql(SQL_DB_TABLE_NAME2, pool, if_exists="replace", index=False, dtype=column_data_types)


# close_conn_to_sql(pool, connector)

benefits.benefitRatingDecimal       float64
benefits.comments                   float64
benefits.highlights                 float64
benefits.numRatings                   int64
benefits.employerSummary             object
                                     ...   
salary.currency.positiveTemplate     object
salary.currency.symbol               object
salary.lastSalaryDate                object
salary.salaries                     float64
wwfu                                float64
Length: 163, dtype: object


     id  index                                wwfu_val_videos_val
0  4327      0  <iframe width="560" height="315" src="https://...
1  4328      0  <iframe width="560" height="315" src="https://...
2  4329      0  <iframe width="560" height="315" src="https://...
3  4330      0  <iframe width="560" height="315" src="https://...
4  4331      0  <iframe width="560" height="315" src="https://...
5  4332      0  <iframe width="560" height="315" src="https://...
6  4333      0  <iframe width="560" height="315" src="https://...
7  4334      0  <iframe width="560" height="315" src="https://...
8  4335      0  <iframe width="560" height="315" src="https://...
9  4336      0  <iframe width="560" height="315" src="https://...
                 id         index
count  17687.000000  17687.000000
mean    8815.499463      0.003732
std     5087.296180      0.060974
min        1.000000      0.000000
25%     4413.500000      0.000000
50%     8815.000000      0.000000
75%    13223.500000      0.000000
ma

In [112]:
# df_w["wwfu_val_body"][86114]
# df_rvr["reviews_val_reviewResponses_val_responseText"][54]

'hello we are glad to know that you enjoy your experience at deezer and we take into account your opinion on the points you are addressing we care about providing the best experience for our employees so we sincerely thank you for the transparency of your feedback keep serving good vibes'